# ManiSDP Python

In [1]:
import os
import numpy as np
import scipy.sparse

try:
    import manisdp
except ImportError:
    !pip install git+https://github.com/husisy/ManiSDP-matlab.git@zc-dev

from manisdp import generate_theta_example, unittrace_cvxpy, manisdp_unittrace, manisdp_onlyunitdiag, read_maxcut_Gset

## Example: `manisdp_onlyunitdiag()`

max-cut problem

adopted from `example/example_maxcut.m` [link](https://github.com/wangjie212/ManiSDP-matlab/blob/main/example/example_maxcut.m)

`data/Gset/G32.txt` in the repo [link](https://github.com/wangjie212/ManiSDP-matlab/blob/main/data/Gset/G32.txt).

$$ \begin{align*}\min_{X}&\;\mathrm{Tr}[CX]\\
\mathrm{s.t.}&\begin{cases}
X\succeq0\\
X_{ii}=1
\end{cases}\end{align*} $$

In [4]:
import os
if not os.path.exists('G32.txt'):
    !wget https://raw.githubusercontent.com/wangjie212/ManiSDP-matlab/refs/heads/main/data/Gset/G32.txt

In [2]:
z0 = read_maxcut_Gset('G32.txt')
matC = -0.25 * scipy.sparse.csc_matrix(z0)
return_info, Y = manisdp_onlyunitdiag(matC, p0=40)
print('cost:', return_info['cost'])
X = Y.T @ Y


Iter 1, obj:-1567.62048669, dinf:2.3e-05, r:17, Y.shape:(40, 2000), time:1.35s
(25, 2000) -1567.6204866937167 -1567.621070899212
Iter 2, obj:-1567.63960060, dinf:6.8e-07, r:10, Y.shape:(25, 2000), time:3.72s
(15, 2000) -1567.6396005951624 -1567.6395944339545
Iter 3, obj:-1567.63964122, dinf:3.4e-08, r:9, Y.shape:(15, 2000), time:5.74s
(13, 2000) -1567.6396412221695 -1567.6395756051998
Iter 4, obj:-1567.63964467, dinf:2.7e-09, r:9, Y.shape:(13, 2000), time:7.73s
Optimality is reached!
cost: -1567.6396446740375


## Example: `manisdp_unittrace()`

theta example?

adopted from `example/example_theta.m` [link](https://github.com/wangjie212/ManiSDP-matlab/blob/main/example/example_theta.m)

$$ \begin{align*} \min_{X}&\mathrm{Tr}[CX]\\
\mathrm{s.t.}&\begin{cases}
X\succeq0\\
\mathrm{Tr}[A_{i}X]=b_{i}
\end{cases} \end{align*} $$

In [3]:
C, A, b = generate_theta_example(n=50)
X_cvxpy, fval_cvxpy = unittrace_cvxpy(C, A.toarray().reshape(-1, *C.shape), b)
kwargs = dict(sigma0=1, tol=1e-8, TR_maxinner=20, TR_maxiter=4, tao=1e-3, theta=1e-2, delta=10, line_search=True, alpha=0.01)
return_info, Y = manisdp_unittrace(C, A, b, **kwargs)

print('cvxpy obj:', fval_cvxpy)
print('manisdp obj:', return_info['cost'])


Iter 1, obj:-49.99971896, gap:3.6e-03, pinf:3.0e-01, dinf:2.4e-16, gradnorm:6.6e-09, r:1, Y.shape:(50, 1), time:0.00s
Iter 2, obj:-49.99745388, gap:1.1e-02, pinf:3.0e-01, dinf:1.3e-15, gradnorm:5.9e-10, r:1, Y.shape:(50, 2), time:0.01s
Iter 3, obj:-49.98594910, gap:2.5e-02, pinf:3.0e-01, dinf:0.0e+00, gradnorm:1.7e-09, r:1, Y.shape:(50, 2), time:0.01s
Iter 4, obj:-49.93362407, gap:5.4e-02, pinf:2.9e-01, dinf:2.2e-16, gradnorm:1.4e-09, r:1, Y.shape:(50, 1), time:0.01s
Iter 5, obj:-49.69829220, gap:1.1e-01, pinf:2.9e-01, dinf:1.4e-16, gradnorm:2.6e-09, r:1, Y.shape:(50, 2), time:0.02s
Iter 6, obj:-48.54881350, gap:2.3e-01, pinf:2.7e-01, dinf:0.0e+00, gradnorm:5.5e-09, r:1, Y.shape:(50, 2), time:0.02s
Iter 7, obj:-39.92844176, gap:3.6e-01, pinf:1.9e-01, dinf:5.1e-02, gradnorm:7.9e-09, r:1, Y.shape:(50, 1), time:0.04s
Iter 8, obj:-20.26804932, gap:8.7e-02, pinf:5.2e-02, dinf:6.5e-02, gradnorm:9.5e-09, r:2, Y.shape:(50, 2), time:0.10s
Iter 9, obj:-17.64889071, gap:2.6e-02, pinf:2.5e-02, din

WARNING: `manisdp_unittrace` is 10x slower than that matlab version. After doing some profiling, I think the sparse matrix multiplication is the bottleneck.